# Computing the bicharacter of huge spaces of polynomials

This code works only for diagrams without hole like partitions or compositions diagrams. 

In [7]:
%display latex
%runfile compute_character.py

## Quick computation

To obtain the complete character of the subpace generated by $V_\gamma$ and closed by partial derivatives and polarization in the Schur basis, simply use `compute_character`. This function uses by default all implemented optimisations. 

In [9]:
compute_character(Partition([1,1,1]))

s[] # s[1, 1, 1]

In [10]:
compute_character(Partition([3,1]))

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] + s[1, 1] # s[1, 1, 1, 1] + s[2] # s[2, 1, 1] + s[3] # s[1, 1, 1, 1]

We may want to factorise the character in terms of representations of $S_n$. 

In [43]:
n = 4
factorise(compute_character(Partition([3,1])), n)

[3, 1]


s[]

[1, 1, 1, 1]


s[1, 1] + s[3]

[2, 2]


s[1]

[2, 1, 1]


s[1] + s[2]

The function `compute_character` is persistent which means that the results for all the partitions up to the partitions of 5 (except the partition 5 itself) are in memory. Thus when called for those partitions the function read the result in the memory instead of computing ot again. 
If you compute characters for other partitions, they will also be added to the memory.

## A detail example on partition 21

A diagram can be a partition or define as a set of cells given by their cartesian coordinates.

In [5]:
diagram = Diagram([(0,0),(0,1),(1,0)])
diagram

<diagram.Diagram instance at 0x7fd059a48710>

In [7]:
diagram.cells() == Partition([2,1]).cells()

True

We can compute the Vandermonde determinant $V_\gamma$ to a partition or a diagram $\gamma$. 

In [8]:
v = vandermonde(Partition([2,1]))
v

-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

In [9]:
v = vandermonde(diagram)
v

-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

 Then we can have the subspace $S$ generated by $V_\gamma$ and its partials derivatives using `Subspace`. It takes in argument two dictionnaries, one for the generators, here $V_\gamma$, and one for the operators, here the partial derivatives that can be applied to $V_\gamma$.

In [13]:
generator = {v.multidegree() : [v]}
operators = partial_derivatives(v.parent())
S = Subspace(generator, operators)
S.basis()

{(0,): (theta01 - theta02, theta00 - theta02),
 (1,): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

We now want to project $S$ on the isotypic components. The result will be a dictionnary of subspaces, each one corresponding to one isotypic component associated to the corresponding partition. The function `IsotypicComponent` needs a subspace and the symmetric group order $n$.

In [19]:
S_iso = IsotypicComponent(S, 3)
print S_iso

{[1, 1, 1]: <subspace.Subspace object at 0x7fd05796ff10>, [2, 1]: <subspace.Subspace object at 0x7fd057bc3a10>}


In [17]:
for subspace in S_iso.values():
    show(subspace.basis())

{((1,),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

{((0,), (2, 1)): (-theta00 + theta02,)}

Then we polarize the isotypic component using `PolarizationSpace` but first we need to define a set of operators for example the polarization operators for 2 sets of variables. 

In [20]:
polarizators = polarization_operators(2)
S_pol = PolarizedSpace(S_iso, polarizators)
print S_pol

{[1, 1, 1]: <subspace.Subspace object at 0x7fd05796f810>, [2, 1]: <subspace.Subspace object at 0x7fd057bd4ed0>}


In [21]:
for subspace in S_pol.values():
    show(subspace.basis())

{((0, 1),
  (1,
   1,
   1)): (-x11*theta00 + x12*theta00 + x10*theta01 - x12*theta01 - x10*theta02 + x11*theta02,),
 ((1, 0),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

{((0, 0), (2, 1)): (theta00 - theta02,)}

From this polarized isotypic components, we can obtain the character. 

In [22]:
character(S_pol)

s[] # s[2, 1] + s[1] # s[1, 1, 1]

## Other possible options

If nothing is specified, the determinant $V_\gamma$ is a polynomial oven one set of variables $x$ and inert variables $\theta$. 

In [24]:
v = vandermonde(Partition([2,1]))
v.parent()

Diagonal Polynomial Ring with 1 rows of 3 variables and inert variables over Rational Field

But it is possible to specify a number of sets of variables for the ring in which $V_\gamma$ lies. At first sight, it change nothing on the determinant it self but now the parent is different. 

In [25]:
v2 = vandermonde(Partition([2,1]),2)
show(v2)
v2.parent()

-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

Diagonal Polynomial Ring with 2 rows of 3 variables and inert variables over Rational Field

When projecting on the isotypic component, it is possible to ask for a specific component, for exemple the alternating one in the case of the partition 3. 

In [31]:
v = vandermonde(Partition([3]))
S = Subspace({v.multidegree(): [v]}, partial_derivatives(v.parent()))
S_3 = IsotypicComponent(S, Partition([1,1,1]))
S_3.basis()

{((3,),
  (1,
   1,
   1)): (-x00^2*x01 + x00*x01^2 + x00^2*x02 - x01^2*x02 - x00*x02^2 + x01*x02^2,)}

And we can polarize only this component. 

In [34]:
S_3pol = PolarizedSpace(S_3, polarization_operators(2))
S_3pol.basis()

{((0, 3),
  (1,
   1,
   1)): (x10^2*x11 - x10*x11^2 - x10^2*x12 + x11^2*x12 + x10*x12^2 - x11*x12^2,),
 ((1, 2),
  (1,
   1,
   1)): (-1/2*x01*x10^2 + 1/2*x02*x10^2 - x00*x10*x11 + x01*x10*x11 + 1/2*x00*x11^2 - 1/2*x02*x11^2 + x00*x10*x12 - x02*x10*x12 - x01*x11*x12 + x02*x11*x12 - 1/2*x00*x12^2 + 1/2*x01*x12^2,),
 ((2, 1),
  (1,
   1,
   1)): (-x00*x01*x10 + 1/2*x01^2*x10 + x00*x02*x10 - 1/2*x02^2*x10 - 1/2*x00^2*x11 + x00*x01*x11 - x01*x02*x11 + 1/2*x02^2*x11 + 1/2*x00^2*x12 - 1/2*x01^2*x12 - x00*x02*x12 + x01*x02*x12,),
 ((3, 0),
  (1,
   1,
   1)): (x00^2*x01 - x00*x01^2 - x00^2*x02 + x01^2*x02 + x00*x02^2 - x01*x02^2,)}

Other operators are implemented such as Steenrod operators of degree $k$ on $r$ sets of operators. 

In [35]:
r = 2
k = 1
steenrod_operators(r, k)

{(-1, 0): [*.steenrod_op(i=0, row_symmetry=None, k=2)],
 (0, -1): [*.steenrod_op(i=1, row_symmetry=None, k=2)]}

But any dictionnary of operators could be used. The keys of the dictionnary simply have to correspond the multidegress of the operators. 

In [37]:
d = merge(polarization_operators(2), steenrod_operators(2,2))
d

{(-2, 0): [*.steenrod_op(i=0, row_symmetry=None, k=3)],
 (-1, 1): [*.polarization(i1=0, row_symmetry=None, i2=1, d=1)],
 (0, -2): [*.steenrod_op(i=1, row_symmetry=None, k=3)],
 (1, -1): [*.polarization(i1=1, row_symmetry=None, i2=0, d=1)]}

In the function `character`, we can specify bases for $GL_r$ and $S_n$. 

In [41]:
v = vandermonde(Partition([2,1]))
S_iso = IsotypicComponent(Subspace({v.multidegree(): [v]}, partial_derivatives(v.parent())), 3)
S_pol = PolarizedSpace(S_iso, polarization_operators(2))
character(S_pol, right_basis=s, left_basis=m)

m[] # s[2, 1] + m[1] # s[1, 1, 1]

## Other example : diagram (0,1,3)

In [49]:
v = vandermonde(Diagram([(0,0),(1,0),(3,0)]))
v.factor()

(x01 - x02) * (-x00 + x01) * (x00 - x02) * (x00 + x01 + x02)

We create the subspace generated by $V_\gamma$ and its partial derivatives and add the Steenrod operators of degree 1 and 2.

In [50]:
generator = {v.multidegree() : [v]}
operators = merge(merge(partial_derivatives(v.parent()), steenrod_operators(1, 1)), steenrod_operators(1, 2))

In [51]:
S = Subspace(generator, operators)
S.basis()

{(0,): (1,),
 (1,): (x00, x01, x02),
 (2,): (x00^2 - x02^2, x01^2 - x02^2, -x00*x01 + x00*x02, -x00*x01 + x01*x02),
 (3,): (1/3*x00^3 - 1/3*x01^3 - x00*x02^2 + x01*x02^2,
  -1/3*x00^3 + x00*x01^2 - x01^2*x02 + 1/3*x02^3,
  -x00^2*x01 + 1/3*x01^3 + x00^2*x02 - 1/3*x02^3),
 (4,): (-x00^3*x01 + x00*x01^3 + x00^3*x02 - x01^3*x02 - x00*x02^3 + x01*x02^3,)}

We can ask for the character before polarization. 

In [52]:
character(IsotypicComponent(S, 3))

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Apply $\sum_i \partial_{x_i}$ and $\sum_i x_i \partial_{x_i}^2$ t $V_{\gamma}$. 

In [53]:
v.symmetric_derivative((1,))

-3*x00^2*x01 + 3*x00*x01^2 + 3*x00^2*x02 - 3*x01^2*x02 - 3*x00*x02^2 + 3*x01*x02^2

In [54]:
v.steenrod_op(0, 2)

-6*x00^2*x01 + 6*x00*x01^2 + 6*x00^2*x02 - 6*x01^2*x02 - 6*x00*x02^2 + 6*x01*x02^2

Polarization

In [55]:
r=2
polarizators = polarization_operators(r)
S_pol = PolarizedSpace(IsotypicComponent(S, 3), polarizators)
character(S_pol)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

This example is one of the few for which the quotien works properly. We first compute the character of the image of `S_pol` by the symmetric derivatives. 

In [59]:
list_degrees = [tuple(k1 if j==i1 else 0 for j in range(r)) for k1 in range(1, 3+1) for i1 in range(0, r)]
symmetric_derivatives = symmetric_derivatives(list_degrees)
character(Range(S_pol, symmetric_derivatives))

s[] # s[3] + s[1] # s[2, 1] + s[2] # s[2, 1] + s[3] # s[1, 1, 1]

And now the difference. 

In [60]:
charac = character(S_pol) - character(Range(S_pol, symmetric_derivatives))
charac

s[1] # s[3] + s[2] # s[2, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Or in terms of monomial symmetric functions.

In [62]:
character(S_pol, left_basis=m) - character(Range(S_pol, symmetric_derivatives), left_basis=m)

m[1] # s[3] + m[1, 1] # s[2, 1] + m[2] # s[2, 1] + m[2, 1] # s[2, 1] + m[2, 2] # s[1, 1, 1] + m[3] # s[2, 1] + m[3, 1] # s[1, 1, 1] + m[4] # s[1, 1, 1]